# Reachy kinematics

In this notebook we will talk about kinematics!

As explained in other notebooks, you can change with the SDK the position of the joints if and only if the motors are in stiff mode. There are two ways to send a new target position: 

* **goal_position**: when this attribute is set, the joint will try to reach it as fast as it can and it can be pretty fast (up to 700 degrees per second). You can lower the *speed_limit* but we don't recommend to use this method as it gets really hard to follow complex trajectories and have precise timing. Even though you can change the motor speed, you have no control over the acceleration.  


* **goto()**: instead of playing with the goal_position you can use the *goto* method that is implemented in the SDK. This method generates a trajectory between the present position and the goal position. This trajectory is then interpolated at a predefined frequency (100Hz) to compute all intermediary target position that should be followed before reaching the final goal position. Depending on the interpolation mode chosen, you can have a better control over speed and acceleration.

The two interpolation modes that we use consist in working either linearly or with the minjerk function.

<img src="https://raw.githubusercontent.com/pollen-robotics/reachy-sdk/notebooks_documentation/notebooks/images/kinematics/interpolation.png?token=AHZJ5IBL7QZHESYVASSGUEDAMG7FU" alt="drawing" width="500"/>

Both trajectories start and finish at the same point but don't follow the same intermediate positions.
The minimum jerk will slowly accelerate at the begining and slowly decelerate at the end. This make the movements way more natural, you'll see this below.

Now let's make our robot move! We will make the same arm movement but in three different ways, by setting the *goal_position* and using the *goto* method with each interpolation mode.

We will put Reachy's right arm in the right-angled position. First, place it on a table and make sure that there are no obstacles along its way.

In [1]:
from reachy_sdk import ReachySDK

from reachy_sdk.trajectory.interpolation import InterpolationMode

In [2]:
reachy = ReachySDK(host='localhost')

In [3]:
# Reachy's right arm joints are joints 8 to 15
for joint in reachy.joints[8:16]:
    print(joint.name)

r_shoulder_pitch
r_shoulder_roll
r_arm_yaw
r_elbow_pitch
r_forearm_yaw
r_wrist_pitch
r_wrist_roll
r_gripper


### goal_position

First, we will use set the *goal_psoition* on each joint to put the right arm in a right-angled position. 

In [3]:
for joint in reachy.joints[8:16]:
    joint.compliant = False

# Define what the goal_position of each joint will be
right_angle_position = {
    'r_shoulder_pitch': 0,
    'r_shoulder_roll': 0,
    'r_arm_yaw': 0,
    'r_elbow_pitch': -90,
    'r_forearm_yaw': 0,
    'r_wrist_pitch': 0,
    'r_wrist_roll': 0,
    'r_gripper': 0,
}

#Set each goal_position
from operator import attrgetter

for name, goal in zip(right_angle_position.keys(), right_angle_position.values()):
    joint = attrgetter(name)(reachy)
    joint.goal_position = goal

A bit too fast, isn't it? The problem with this is that you can't control the movement's speed.

Now we will put the joints back to compliant mode, so put your hand below the arm to prevent the arm from falling on the table.

In [4]:
for joint in reachy.joints[8:16]:
    joint.compliant = True

### goto with linear interpolation

In [56]:
# First put the joints of the right arm in stiff mode
for joint in reachy.joints[8:16]:
    joint.compliant = False

In [57]:
reachy.goto(
    {getattr(reachy, joint.name): gp for joint, gp in zip(reachy.joints[8:16], harder_pos.values())},
duration=2.0,
interpolation_mode=InterpolationMode.LINEAR,
)

In [58]:
# First put the joints of the right arm in stiff mode
for joint in reachy.joints[8:16]:
    joint.compliant = True

### goto with minjerk interpolation

In [59]:
# First put the joints of the right arm in stiff mode
for joint in reachy.joints[8:16]:
    joint.compliant = False

In [60]:
reachy.goto(
    {getattr(reachy, joint.name): gp for joint, gp in zip(reachy.joints[8:16], harder_pos.values())},
duration=2.0,
interpolation_mode=InterpolationMode.MINIMUM_JERK
)

In [61]:
for joint in reachy.joints[8:16]:
    joint.compliant = True

The use of goto has the advantage of giving control easily on the trajectory time. The difference between the two interpolation modes might not be huge on a simple movement like the on we just did, but on more complexe ones, the difference can be quite important. 

Like we said before, minjerk interpolation gives more natural movements. Try it yourself with custom movements!

You can try with the following goal position for example where Reachy places its right end-effector on its right hip.

In [62]:
harder_pos = {
'r_shoulder_pitch': 16.726945447159054,
 'r_shoulder_roll': -66.43789209449601,
 'r_arm_yaw': 61.054944877528094,
 'r_elbow_pitch': -120.92307608487477,
 'r_forearm_yaw': -12.463343291751162,
 'r_wrist_pitch': -27.824175772826795,
 'r_wrist_roll': 41.20234435672571,
 'r_gripper': -25.073313669509574
}

## Arm coordinate system

### Joint coordinates

In all examples above, we have used what is called joint coordinates to move Reachy. This means that we have controlled each joint separately.

### Kinematic model

Controlling a robot in joint coordinates can be hard and is often far from what we actually do as humans. When we want to grasp an object in front of us, we think of where we should put our hand, not how to flex each individual muscle to reach this position. This approach relies on the cartesian coordinates: the 3D position and orientation in space.

Forward and Inverse Kinematics is a way to go from one coordinates system to the other:

* **forward kinematics: joint coordinates –> cartesian coordinates**
* **inverse kinematics: cartesian coordinates –> joint coordinates**

We have defined the whole kinematic model of the arm. This means the translation and rotation required to go from one joint to the next one. On a right arm equipped with a force gripper this actually look like this:

<img src="https://raw.githubusercontent.com/pollen-robotics/reachy-sdk/notebooks_documentation/notebooks/images/kinematics/right_arm_kinematic_model.png?token=AHZJ5IGJ7XNJFFZHHQTTZODAMHYQG" alt="drawing" width="300"/>

To use and understand the kinematic model, you need to know how Reachy coordinate system is defined (from Reachy's perspective), see below:

<img src="https://raw.githubusercontent.com/pollen-robotics/reachy-sdk/notebooks_documentation/notebooks/images/kinematics/arm_axis.png?token=AHZJ5ID66GQKUY3FBQ74GR3AMHNIU" alt="drawing" width="300"/>

* the X axis corresponds to the foward arrow.
* the Y axis corresponds to the right to left arrow.
* the Z axis corresponds to the up arrow.

The origin of this coordinate system is located in the upper part of the robot trunk. Basically, if you imagine a segment going from the left shoulder to the right shoulder of the robot, the origin is the middle of this segment.

The units used for this coordinate system are the meter. So the point (0.3, -0.2, 0) is 30cm in front of the origin, 20cm to the right and at the same height.

## Forward kinematics

Using the joint coordinates and the kinematic model defined above, we can compute the 3D position and orientation of the right or left end-effector with the *forward_kinematics* method.

We consider the end-effector to be in a virtual joint located in the gripper and referred as *'right_tip'* or *'left_tip'* in the [urdf file](https://github.com/pollen-robotics/reachy_kinematics/blob/master/reachy.URDF), as shown below. 

<img src="https://raw.githubusercontent.com/pollen-robotics/reachy-sdk/notebooks_documentation/notebooks/images/kinematics/eef.png" alt="drawing" width="200"/>

The origin of the frame corresponds to the *'right_tip'*.

You can see the right and left end-effectors animated in the gif below.

<img src="https://raw.githubusercontent.com/pollen-robotics/reachy-sdk/notebooks_documentation/notebooks/images/safety/reachy_eef.gif" alt="drawing" width="300"/>

In [20]:
reachy.r_arm.forward_kinematics()

array([[ 0.07297327,  0.04151189, -0.9964696 ,  0.48141822],
       [ 0.13201542,  0.98993957,  0.05090758, -0.21421645],
       [ 0.98855796, -0.13526424,  0.06675892, -0.30646518],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

The 4x4 matrix returned by the *forward_kinematics* method is what is often called a pose. It actually encodes both the 3D translation (as a 3D vector) and the 3D rotation (as a 3x3 matrix) into one single representation. 

\begin{bmatrix}
R_{11} & R_{12} & R_{13} & T_{x} \\
R_{21} & R_{22} & R_{23} & T_{y} \\
R_{31} & R_{32} & R_{33} & T_{z} \\
0 & 0 & 0 & 1 \\
\end{bmatrix}

The instruction 

```python
reachy.r_arm.forward_kinematics()
```

returns the current pose of the right end-effector, based on the present position of every joint in the right arm. 

You can also compute the pose for a given configuration, to do that just pass the list of position as argument of *forward_kinematics*. Be careful to respect the order of the position you give.

For example, we can compute the forward kinematics for the right-angle position we used earlier.

In [5]:
reachy.r_arm.forward_kinematics(list(right_angle_position.values())[:-1])

array([[ 0.    ,  0.    , -1.    ,  0.3675],
       [ 0.    ,  1.    ,  0.    , -0.202 ],
       [ 1.    ,  0.    ,  0.    , -0.28  ],
       [ 0.    ,  0.    ,  0.    ,  1.    ]])

With this result, we can tell that when the right arm is in the right angle position, the right end-effector is 37cm in front of the origin, 20cm to the left and 28cm below the origin.the same height.

The rotation matrix 

\begin{bmatrix}
0 & 0 & -1\\
0 & 1 & 0\\
1 & 0 & 0\\
\end{bmatrix}

means that the hand is facing forward.

A rotation matrix equal to the identity matrix 

\begin{bmatrix}
1 & 0 & 0\\
0 & 1 & 0\\
0 & 0 & 1\\
\end{bmatrix}

means that the hand is facing toward the bottom.

## Inverse kinematics

In [6]:
reachy.r_arm.inverse_kinematics(reachy.r_arm.forward_kinematics(list(right_angle_position.values())[:-1]))

[0.0, 0.0, 0.0, -90.0, 0.0, 0.0, 0.0]

Knowing where you arm is located in the 3D space can be useful but most of the time what you want is to move the arm in cartesian coordinates. You want to have the possibility to say: “move your hand to [x, y, z] with a 90° rotation around the Z axis”.

This is what inverse kinematics does. We have provided a method to help you with that. You need to give it a 4x4 target pose (as given as a result by the forward kinematics) and an initial joint state.

To make this more concrete, let's first try with a simple example. We will make the hand of the robot goes from a point A to a point B in 3D space. You always have to provide poses to the inverse kinematics that are actually reachable by the robot.

For our starting point, let's imagine a point in front of the robot rigth shoulder and slightly below. If you remember Reachy coordinate system, we can define such a point with the following coordinates:

\begin{equation} A =
\begin{pmatrix}
0.3 & -0.2 & -0.3
\end{pmatrix}
\end{equation}

For our end point we will stay in a parallel plan in front of the robot (we keep the same x) and move to the upper left (20cm up and 20cm left). This gives us a B point such that:

\begin{equation} B =
\begin{pmatrix}
0.3 & 0.0 & -0.1
\end{pmatrix}
\end{equation}

But having the 3D position is not enough to design a pose. You also need to provide the 3D orientation. The identity rotation matrix corresponds to the zero position of the robot, ie. when the hand is facing toward the bottom. So if we want the hand facing forward when going from A to B, we need to rotate it from -90° around the y axis.